In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import os

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from src.utils.time_series_procs import split_sequences
from src.models.transformer_ts_tf import Transformer

In [2]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [3]:
def get_baseline_model(
    num_features, time_steps, num_heads=2, num_layers_enc=1, num_feed_forward=32
):
    model = Transformer(
        num_hid=num_features,
        time_steps=time_steps,
        num_head=num_heads,
        num_layers_enc=num_layers_enc,
        num_feed_forward=num_feed_forward,
    )

    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(
        optimizer=opt,
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [4]:
def make_repeated_time_series_k_fold(
    x,
    y,
    train_period,
    test_period,
    repeats=10,
    show_individual_results=True,
    show_final_result=True,
):
    results = []
    max_samples = x.shape[0]

    for _ in range(repeats):
        i = 0
        scores = []
        for _ in range(0, max_samples // train_period):
            x_train = x[i : i + train_period]
            y_train = y[i : i + train_period]
            x_test = x[i + train_period : i + train_period + test_period]
            y_test = y[i + train_period : i + train_period + test_period]
            i += train_period
            x_train = scaler.fit_transform(
                x_train.reshape(-1, x_train.shape[-1])
            ).reshape(x_train.shape)
            x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(
                x_test.shape
            )
            model = get_baseline_model(
                num_features=x_train.shape[-1], time_steps=x_train.shape[-2]
            )
            model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
            rmse = model.evaluate(x_test, y_test, verbose=0)
            scores.append(rmse[1])
        results.append(scores)
        if show_individual_results:
            print("RMSE: %.3f (%.3f)" % (np.mean(scores), np.std(scores)))
    if show_final_result:
        print("\nRMSE: %.3f (%.3f)" % (np.mean(results), np.std(results)))
    return results

In [5]:
def make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats=10, timesteps_list=[3], show_results=True
):
    results = {}

    for timesteps in timesteps_list:
        x, y = split_sequences(df.values, timesteps)
        scores = make_repeated_time_series_k_fold(
            x,
            y,
            train_period,
            test_period,
            repeats=10,
            show_individual_results=False,
            show_final_result=False,
        )
        results[timesteps] = scores
        if show_results:
            print(
                "TIMESTEPS: %d RMSE: %.3f (%.3f)"
                % (timesteps, np.mean(scores), np.std(scores))
            )
    return results

<h3>Disable Tensorflow Warnings</h3>

In [6]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [7]:
df = pd.read_csv("../../../../data/interim/partner_iii/Dati CADD 2020-2022_with_fillna.csv")

In [8]:
seed = 47
scaler = StandardScaler()

timesteps_list = [1, 3, 5, 7, 10, 15, 20]
repeats = 10

# Transformer Encoder

# Configs:

1. TIMESTEPS: [1, 3, 5, 7, 10, 15, 20]

2. Epochs: 100

3. Batch size: 32

4. num heads: 2

5. FF Layers enc: 1

      5.1 Units 32

6. Optimization Algorithm: Adam
    
    6.1. Learning Rate: 1e-2

# Times Series Repeated KFold Cross validation - different Timesteps values

Here we will make a Times Series KFold Cross validation. We are using the Blocking Time Series Split method. And with timeseries Split method. We also do it with different number of timesteps


<h3> Time Series Repeated KFold Cross Validation with vary timesteps</h3>

<h4>Experiment 1</h4>
    
<b>Train 80 of that data and test with 20%</b>

In [10]:
train_period = 200
test_period = 30

In [13]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 5.847 (0.762)
TIMESTEPS: 3 RMSE: 6.162 (1.411)
TIMESTEPS: 5 RMSE: 6.423 (2.168)
TIMESTEPS: 7 RMSE: 6.843 (2.526)
TIMESTEPS: 10 RMSE: 7.304 (2.746)
TIMESTEPS: 15 RMSE: 6.759 (1.909)
TIMESTEPS: 20 RMSE: 7.077 (2.336)
